# Conjunto de Dados 2: *Shezhen Dataset*
***
* Disponível em: <https://www.kaggle.com/kmader/pulmonary-chest-xray-abnormalities>. Acesso em 19 fev. 2021.

### Importação dos pacotes

In [2]:
from keras.preprocessing.image import ImageDataGeneratorimport glob
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from tqdm import tqdm
import urllib.request
import pandas as pd
import numpy as np
import re

### Pré-processamento dos Dados

In [4]:
path_file = '../0-datasets/montgomery-shenzen/arquivos-descompactados/ChinaSet_AllFiles/ChinaSet_AllFiles/CXR_png/*.png'
filelist_shenzen = glob.glob(path_file)

print('quantidade de imagens:', str(len(filelist_shenzen)))

quantidade de imagens: 662


In [5]:
def extract_label(file_list):
  '''
  extraindo os rótulos da imagem

  Args:
    file_list (list) --> lista com a referência dos arquivos de imagem
  
  Returns:
    labels (list) --> lista com as rotulações da imagem
  '''

  labels = list()
    
  for file in tqdm(file_list): 
    current_label = re.findall('[0-9]{4}_(.+?).png', file) 
    if current_label[0] == '0':
      labels.append('normal')
    else:
      labels.append('abnormal')
        
  return labels

In [7]:
labels = extract_label(filelist_shenzen)
print('quantidade de rótulos:', str(len(labels)))

quantidade de rótulos: 662


In [11]:
full_data = pd.DataFrame(filelist_shenzen, columns = ['filepath'])
full_data['target'] = labels
train_df, test_df = train_test_split(full_data, stratify = full_data['target'], test_size = 0.2, random_state = 42)
train_df, validation_df = train_test_split(train_df, stratify = train_df['target'], test_size = 0.2, random_state = 42)

print('quantidade de imagens de treinamento:', len(train_df['filepath']))
print('quantidade de rótulos de treinamento:', len(train_df['target']))
print('quantidade de imagens de teste:', len(test_df['filepath']))
print('quantidade de rótulos de teste:', len(test_df['target']))
print('quantidade de imagens de validação:', len(validation_df['filepath']))
print('quantidade de rótulos de validação:', len(validation_df['target']))

quantidade de imagens de treinamento: 423
quantidade de rótulos de treinamento: 423
quantidade de imagens de teste: 133
quantidade de rótulos de teste: 133
quantidade de imagens de validação: 106
quantidade de rótulos de validação: 106


In [13]:
class_weights = class_weight.compute_class_weight('balanced', np.unique(train_df['target']), train_df['target'])
class_weight = {0: class_weights[0], 1: class_weights[1]}

train_df.to_csv('train_df.csv')
validation_df.to_csv('validation_df.csv')
test_df.to_csv('test_df.csv')

### Gerador de dados para a rede pelo Keras

In [14]:
image_generator = ImageDataGenerator(samplewise_center = True, samplewise_std_normalization = True,
                                     rotation_range = 20, zoom_range = 0.2)

train_generator = image_generator.flow_from_dataframe(dataframe = train_df, directory = '', x_col = 'filepath',
                                                      y_col = 'target', batch_size = 32, seed = 42, shuffle = True,
                                                      class_mode = 'categorical', target_size = (256, 256))

valid_generator = image_generator.flow_from_dataframe(dataframe = validation_df, directory = '.', x_col = 'filepath',
                                                      y_col = 'target', batch_size = 32, seed = 42,
                                                      shuffle = True, class_mode = 'categorical',
                                                      target_size = (256, 256))

test_datagen = ImageDataGenerator(samplewise_center = True, samplewise_std_normalization = True)

test_generator = test_datagen.flow_from_dataframe(dataframe = test_df, directory = '.', x_col = 'filepath',
                                                  y_col = 'target', batch_size = 32, seed = 42,
                                                  shuffle = True, class_mode = 'categorical',
                                                  target_size = (256, 256))

Found 423 validated image filenames belonging to 2 classes.
Found 106 validated image filenames belonging to 2 classes.
Found 133 validated image filenames belonging to 2 classes.


In [15]:
train_generator.class_indices

{'abnormal': 0, 'normal': 1}